In [1]:

!pip install kaggle
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c nlp-getting-started  # Corrected competition name


!unzip nlp-getting-started.zip

Saving kaggle.json to kaggle.json
100% 593k/593k [00:00<00:00, 730kB/s]
100% 593k/593k [00:00<00:00, 729kB/s]
Archive:  nlp-getting-started.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.metrics import accuracy_score
import pandas as pd
import torch.nn.functional as F
import re
import nltk
from nltk.corpus import stopwords
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train_df = train_df[['id','text','target']]
test_df = test_df[['id','text']]
train_df.isna().sum()
test_df.isna().sum()

,0
id,0
text,0


In [5]:
train_df.set_index('id', inplace=True)
train_df.head()

,text,target
id,,
1,Our Deeds are the Reason of this #earthquake M...,1
4,Forest fire near La Ronge Sask. Canada,1
5,All residents asked to 'shelter in place' are ...,1
6,"13,000 people receive #wildfires evacuation or...",1
7,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text):
    text = text.lower()
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

def lemmatize_text(text):
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags
    ]
    return ' '.join(lemmatized_tokens)

def preprocess_text(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    # text = lemmatize_text(text)
    return text

train_df['text'] = train_df['text'].apply(preprocess_text)
test_df['text'] = test_df['text'].apply(preprocess_text)
train_df.head()

,text,target
id,,
1,deeds reason earthquake may allah forgive us,1
4,forest fire near la ronge sask canada,1
5,residents asked shelter place notified officer...,1
6,13000 people receive wildfires evacuation orde...,1
7,got sent photo ruby alaska smoke wildfires pou...,1


In [7]:
all_text = ' '.join(train_df['text']) + " " + ' '.join(test_df['text'])
vocab = {word:index for index,word in enumerate(set(all_text.split()))}
vocab['<unk>'] = len(vocab)
vocab['<pad>'] = len(vocab)
print(len(vocab))


22694


In [8]:
test_df['target'] = 0
test_df.head()

,id,text,target
0,0,happened terrible car crash,0
1,2,heard earthquake different cities stay safe ev...,0
2,3,forest fire spot pond geese fleeing across str...,0
3,9,apocalypse lighting spokane wildfires,0
4,11,typhoon soudelor kills 28 china taiwan,0


In [45]:
class DisasterTweetsDataset(Dataset):
    def __init__(self,data_list,vocab,pad_token='<pad>'):
      self.data = data_list
      self.vocab = vocab
      self.pad_token = pad_token

    def __len__(self):
      return len(self.data)

    def __getitem__(self,idx):
      label,text = self.data[idx]
      text_indices = [self.vocab.get(word,self.vocab['<unk>']) for word in text.split()]
      label = torch.tensor(label, dtype = torch.float32)
      return label, text_indices
    def collate_fn(self,batch):
      labels,texts = zip(*batch)
      texts = [torch.tensor(text,dtype=torch.long) for text in texts]
      texts = nn.utils.rnn.pad_sequence(texts,batch_first=True,padding_value = self.vocab[self.pad_token])
      labels = torch.tensor(labels,dtype=torch.float32)
      return texts.to(device),labels.to(device)

train_size = int(0.7*len(train_df))
val_size = len(train_df) - train_size
train_data,val_data= random_split(train_df.values, [train_size,val_size])
train_df1 = train_df.iloc[train_data.indices]
val_df = train_df.iloc[val_data.indices]
train_data_list = list(zip(train_df1['target'],train_df1['text']))
val_data_list = list(zip(val_df['target'],val_df['text']))
test_data_list = list(zip(test_df['target'],test_df['text']))
train_dataset = DisasterTweetsDataset(train_data_list,vocab)
val_dataset = DisasterTweetsDataset(val_data_list,vocab)
test_dataset = DisasterTweetsDataset(test_data_list,vocab)
train_dataloader = DataLoader(train_dataset,batch_size=6,shuffle=True,collate_fn=train_dataset.collate_fn,drop_last=True)
val_dataloader = DataLoader(val_dataset,batch_size=6,shuffle=False,collate_fn=val_dataset.collate_fn,drop_last=True)
test_dataloader = DataLoader(test_dataset,batch_size=6,shuffle=False,collate_fn=test_dataset.collate_fn,drop_last=False)


In [ ]:
class FakeGRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout_prob, pretrained_embeddings=None):
        super(FakeGRU, self).__init__()

        # Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # Bidirectional GRU
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, dropout=dropout_prob)
        # Attention Layer
        self.attention = nn.Linear(hidden_dim * 2, 1)  # For bidirectional GRU
        # Fully Connected Layers
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # Dropout
        self.dropout = nn.Dropout(dropout_prob)
        # Activation
        self.activation = nn.ReLU()
        # Layer Normalization
        self.layernorm = nn.LayerNorm(hidden_dim * 2)
        # Weight Initialization
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name and param.dim() > 1:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)

    def forward(self, text):
        embed = self.embedding(text)
        gru_out, _ = self.gru(embed)
        attention_weights = torch.softmax(self.attention(gru_out), dim=1)
        context_vector = torch.sum(attention_weights * gru_out, dim=1)
        context_vector = self.layernorm(context_vector)
        context_vector = self.dropout(context_vector)
        hidden = self.fc1(context_vector)
        hidden = self.activation(hidden)
        hidden = self.dropout(hidden)
        return self.fc2(hidden)

vocab_size = len(vocab)
embedding_dim = 128
hidden_dim = 128
output_dim = 1
dropout_prob = 0.7

model = FakeGRU(vocab_size, embedding_dim, hidden_dim, output_dim, dropout_prob).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005, weight_decay=1e-3)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)


def train_model(model,dataloader,optimizer,criterion,max_grad_norm=1.0):
  model.train()
  total_loss = 0
  for texts, labels in dataloader:
    optimizer.zero_grad()
    outputs = model(texts).squeeze(1)
    loss = criterion(outputs,labels)
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(),max_grad_norm)
    optimizer.step()
    total_loss += loss.item()
  return total_loss/len(dataloader)

def evaluate_model(model,dataloader,criterion):
  model.eval()
  total_loss = 0
  all_preds = []
  all_labels = []
  with torch.no_grad():
    for texts,labels in dataloader:
      outputs = model(texts).squeeze(1)
      loss = criterion(outputs,labels)
      total_loss += loss.item()
      preds = torch.round(torch.sigmoid(outputs)).cpu().tolist()
      all_labels += labels.cpu().tolist()
      all_preds += preds
  accuracy = accuracy_score(all_labels,all_preds)
  f1 = f1_score(all_labels,all_preds)
  return total_loss/len(dataloader),accuracy,f1

In [52]:
class FakeLSTM(nn.Module):
    def __init__(self, hidden_dim, embed_dim, vocab_size, output_dim, dropout_prob,num_heads,max_len):
        super(FakeLSTM, self).__init__()


        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # Positional Embedding
        self.positional_embedding = nn.Embedding(max_len, embed_dim)

        # LSTM for sequence processing
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True, dropout=dropout_prob)

        # Multi-Head Self-Attention
        self.multihead_attention = nn.MultiheadAttention(embed_dim=hidden_dim * 2, num_heads=num_heads, dropout=dropout_prob)

        # Fully Connected Layers
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.fc2 = nn.Linear(hidden_dim * 2, output_dim)

        # Residual Connections and Normalization
        self.layernorm1 = nn.LayerNorm(hidden_dim * 2)
        self.layernorm2 = nn.LayerNorm(hidden_dim * 2)

        # Dropout and Activation
        self.dropout = nn.Dropout(dropout_prob)
        self.activation = nn.ReLU()

        # Attention Layer for Context Vector
        self.attention = nn.Linear(hidden_dim * 2, 1)

        # Weight Initialization
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name and param.dim() > 1:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)

    def forward(self, text):
        batch_size, seq_len = text.size()

        # Word Embedding + Positional Embedding
        word_embed = self.embedding(text)  # Shape: (batch_size, seq_len, embed_dim)
        pos_ids = torch.arange(seq_len, device=text.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_embed = self.positional_embedding(pos_ids)  # Shape: (batch_size, seq_len, embed_dim)
        embed = word_embed + pos_embed  # Shape: (batch_size, seq_len, embed_dim)

        # LSTM Processing
        lstm_out, _ = self.lstm(embed)  # Shape: (batch_size, seq_len, hidden_dim*2)

        # Multi-Head Self-Attention
        lstm_out_permuted = lstm_out.permute(1, 0, 2)  # Shape: (seq_len, batch_size, hidden_dim*2)
        attn_output, _ = self.multihead_attention(lstm_out_permuted, lstm_out_permuted, lstm_out_permuted)
        attn_output = attn_output.permute(1, 0, 2)  # Shape: (batch_size, seq_len, hidden_dim*2)

        # Residual Connection + Layer Norm
        attn_output = self.layernorm1(attn_output + lstm_out)

        # Attention Mechanism for Context Vector
        attention_weights = torch.softmax(self.attention(attn_output), dim=1)  # Shape: (batch_size, seq_len, 1)
        context_vector = torch.sum(attention_weights * attn_output, dim=1)  # Shape: (batch_size, hidden_dim*2)

        # Residual Connection + Layer Norm
        residual = context_vector
        context_vector = self.layernorm2(context_vector)
        context_vector = self.dropout(context_vector)

        # Fully Connected Layers
        hidden = self.fc1(context_vector)
        hidden = self.activation(hidden)
        hidden = self.dropout(hidden)
        hidden += residual  # Shape: (batch_size, hidden_dim*2)
        return self.fc2(hidden)


vocab_size = len(vocab)
embedding_dim = 64
hidden_dim = 64
output_dim = 1
dropout_prob = 0.7
num_heads = 16
max_len = train_df['text'].apply(len).max()


model = FakeLSTM(hidden_dim,embedding_dim,vocab_size,output_dim,dropout_prob,num_heads,max_len).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00002, weight_decay=1e-3)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)


def train_model(model,dataloader,optimizer,criterion,max_grad_norm=1.0):
  model.train()
  total_loss = 0
  for texts, labels in dataloader:
    optimizer.zero_grad()
    outputs = model(texts).squeeze(1)
    loss = criterion(outputs,labels)
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(),max_grad_norm)
    optimizer.step()
    total_loss += loss.item()
  return total_loss/len(dataloader)

def evaluate_model(model,dataloader,criterion):
  model.eval()
  total_loss = 0
  all_preds = []
  all_labels = []
  with torch.no_grad():
    for texts,labels in dataloader:
      outputs = model(texts).squeeze(1)
      loss = criterion(outputs,labels)
      total_loss += loss.item()
      preds = torch.round(torch.sigmoid(outputs)).cpu().tolist()
      all_labels += labels.cpu().tolist()
      all_preds += preds
  accuracy = accuracy_score(all_labels,all_preds)
  f1 = f1_score(all_labels,all_preds)
  return total_loss/len(dataloader),accuracy,f1


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.7 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [53]:

best_val_loss = float('inf')
early_stop_counter = 0
patience = 3
num_epochs = 20

for epoch in range(num_epochs):
    train_loss = train_model(model, train_dataloader, optimizer, criterion)
    val_metrics = evaluate_model(model, val_dataloader, criterion)
    val_loss, val_accuracy, val_f1 = val_metrics

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, F1: {val_f1:.4f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "bes_model.pth")
        print("Best model saved!")
        early_stop_counter = 0

    else:
        optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * 0.5
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered!")
            break

    scheduler.step(val_loss)

Epoch 1/20
Train Loss: 1.1771, Val Loss: 0.6783, Val Accuracy: 0.5526, F1: 0.0154
Best model saved!
Epoch 2/20
Train Loss: 1.0181, Val Loss: 0.6698, Val Accuracy: 0.6241, F1: 0.4559
Best model saved!
Epoch 3/20
Train Loss: 0.9040, Val Loss: 0.6619, Val Accuracy: 0.6671, F1: 0.5748
Best model saved!
Epoch 4/20
Train Loss: 0.8425, Val Loss: 0.6431, Val Accuracy: 0.6531, F1: 0.4012
Best model saved!
Epoch 5/20
Train Loss: 0.7846, Val Loss: 0.6350, Val Accuracy: 0.6197, F1: 0.2646
Best model saved!
Epoch 6/20
Train Loss: 0.7225, Val Loss: 0.5830, Val Accuracy: 0.7224, F1: 0.6688
Best model saved!
Epoch 7/20
Train Loss: 0.6355, Val Loss: 0.5342, Val Accuracy: 0.7557, F1: 0.6729
Best model saved!
Epoch 8/20
Train Loss: 0.5183, Val Loss: 0.4811, Val Accuracy: 0.7798, F1: 0.7568
Best model saved!
Epoch 9/20
Train Loss: 0.4101, Val Loss: 0.4532, Val Accuracy: 0.7987, F1: 0.7565
Best model saved!
Epoch 10/20
Train Loss: 0.3391, Val Loss: 0.4823, Val Accuracy: 0.7925, F1: 0.7629
Epoch 11/20
Train

In [54]:
state_dict = torch.load("bes_model.pth", weights_only=True)  # Explicitly set weights_only=True
model.load_state_dict(state_dict)
print("Best model loaded.")

Best model loaded.


In [55]:
model.eval()
preds = []
with torch.no_grad():
    for texts, _ in test_dataloader:
        texts = texts.to(device)
        outputs = model(texts).squeeze(1)
        predicted_labels = torch.round(torch.sigmoid(outputs)).cpu().tolist()
        preds += predicted_labels

test_df['predictions'] = preds

test_df.head()

,id,text,target,predictions
0,0,happened terrible car crash,0,1.0
1,2,heard earthquake different cities stay safe ev...,0,0.0
2,3,forest fire spot pond geese fleeing across str...,0,1.0
3,9,apocalypse lighting spokane wildfires,0,0.0
4,11,typhoon soudelor kills 28 china taiwan,0,1.0


In [56]:
submit_df = test_df[['id','predictions']]

submit_df.columns = ['id','target']
submit_df['target'] = submit_df['target'].astype(int)

submit_df.head()

<ipython-input-56-862d9550651b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit_df['target'] = submit_df['target'].astype(int)


,id,target
0,0,1
1,2,0
2,3,1
3,9,0
4,11,1


In [ ]:
sub = pd.read_csv("sample_submission.csv")
sub.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [57]:
submit_df.to_csv('sbm.csv',index=False)


In [59]:
!kaggle kernels output yosefibrahim/natural-language-processing-with-disaster-tweets -p /path/to/dest

Kernel log downloaded to /path/to/dest/natural-language-processing-with-disaster-tweets.log 
